In [35]:
f = open('input').read().split('\n')

In [36]:
inp = [[int(x) for x in y] for y in f]

In [37]:
def get_col(start, end, inputs):
    column = []
    while start < end:
        column += inputs[start]
        start += 1
    return column

def extend_matrix(inp):
    inputs = []
    inputs.append(inp)
    
    for i in range(10):
        inputs.append([[x + 1 if x + 1 < 10 else 1 for x in y] for y in inputs[i]])

    columns = [get_col(i, i + 5, inputs=inputs) for i in range(5)]
    
    matrix = [columns[0][i] + columns[1][i] + columns[2][i] + columns[3][i] + columns[4][i] for i in range(len(columns[0]))]
    
    return matrix

In [38]:
matrix = extend_matrix(inp=inp)

In [39]:
from dataclasses import dataclass

@dataclass
class Node:
    i: int
    j: int
    v: int = None
    cost: int = 100000
    visited: bool = False
    def __post_init__(self):
        self.v = matrix[self.i][self.j]
        
    def add_cost(self, cost):
        self.cost = min(cost, self.cost) 
        return self
    
    def visit(self):
        self.visited = True
    
    def unvisit(self):
        self.visited = False
    
    def __str__(self):
        return f"Cost to get here: {self.cost}, visited: {self.visited}"
    

In [40]:
def get_neighbour_indices(m, i, j):
    """Get coordinates of neighbours taking into account
    borders of the matrix."""
    permutations = [[i, max(0, j - 1)], [i, min(j + 1, len(m[0]) - 1)], 
                    [max(0, i - 1), j], [min(i + 1, len(m) - 1), j]]
    return [y for y in permutations if y[0] != i or y[1] != j]  

In [41]:
def update_graph(nodes, i, j):
    # Dijkstra algorithm
    current_node = nodes[i][j]
    indices = get_neighbour_indices(nodes, i, j)
    
    neighbours = [nodes[i][j] for i, j in indices]
#     non_visited = [n for n in neighbours if not n.visited]
    
    for node in neighbours:
        if current_node.cost + node.v < node.cost:
            node.add_cost(current_node.cost + node.v)
            
    current_node.visit()
    return nodes


In [42]:
# instantiate graph
nodes = [[Node(i, j) for j in range(len(matrix[0]))] for i in range(len(matrix))]

# set cost for first node to zero
nodes[0][0].add_cost(0)

Node(i=0, j=0, v=7, cost=0, visited=False)

In [31]:
for i in range(len(nodes)):
    for j in range(len(nodes)):
        update_graph(nodes, i, j)

In [51]:
# traverse matrix diagonally to see if it makes a difference
ROW = len(matrix[0])
COL = len(matrix)
for line in range(1, (ROW + COL)):

    start_col = max(0, line - ROW)

    count = min(line, (COL - start_col), ROW)

    for j in range(0, count):
        
        x = min(ROW, line) - j - 1
        y = start_col + j

        update_graph(nodes, x, y)

In [52]:
nodes[-1][-1]

Node(i=499, j=499, v=8, cost=2963, visited=True)